# Complete Vault Management Workflow

This notebook demonstrates a complete workflow with the Laive Python SDK:
- Creating a new vault
- Uploading multiple documents
- Monitoring upload progress
- Querying the vault once documents are processed

> if you only want to run a query, you can use the following notebook : [simple_query_example.ipynb](simple_query_example.ipynb)

## Prerequisites

Before running this notebook, make sure you have:
1. Installed the Laive SDK: `pip install laive-sdk`
2. Set your API key as an environment variable: `LAIVE_API_KEY` or in a `.env` file as in the `.env.example` file
3. Some sample documents in the `./sample_docs` directory (PDF, Images, etc.)

You can get your API key from [https://beta.laive.ai](https://beta.laive.ai)


In [ ]:
import time

# Load environment variables from .env file (optional, if using python-dotenv)
from dotenv import load_dotenv
load_dotenv()

# Initialize the client (API key is read automatically from LAIVE_API_KEY)
from laive import LaiveClient
client = LaiveClient()

# Alternatively, you can initialize the client with a the API key directly
# client = LaiveClient(api_key="your_api_key")


## Step 1: Create a New Vault

Let's create a new vault for our documents:


In [ ]:
print("Creating a new vault...")
vault = client.create_vault(
    name="Research Vault - Notebook Demo",
    description="A vault for my research papers and documents created from laive python sdk"
)

vault_id = vault['id']

print("Vault created successfully!")
print(f"Vault ID: {vault_id}")
print(f"Vault Name: {vault['name']}")
print(f"Description: {vault['description']}")


## Step 2: List All Vaults

Let's verify our vault was created by listing all vaults:


In [ ]:
print("Listing all vaults:")
vaults = client.get_vaults()

for v in vaults:
    status = " (NEW)" if v['id'] == vault_id else ""
    description = v['description'] if v['description'] else "No description"
    print(f"Vault {v['id']}: {v['name']} - {description}{status}")
    
print(f"\nTotal vaults: {len(vaults)}")


## Step 3: Prepare Files for Upload

Let's check what sample documents are available for upload:


In [ ]:
upload_dir = "./sample_docs"  # Directory with sample documents

if not os.path.exists(upload_dir):
    print(f"Sample directory {upload_dir} not found.")
    print("Please create it and add some files for uploading")
else:
    files_to_upload = []
    source_names = []
    
    print(f"Checking directory: {upload_dir}")
    all_files = os.listdir(upload_dir)
    
    for filename in all_files:
        file_path = os.path.join(upload_dir, filename)
        files_to_upload.append(file_path)
        source_names.append(filename)
    
    print(f"\nFound {len(files_to_upload)} files for upload : {files_to_upload}")


## Step 4: Upload Files to Vault

Now let's upload the files to our newly created vault:


In [ ]:
if files_to_upload:
    print(f"Uploading {len(files_to_upload)} files to vault {vault_id}...")
    
    upload_result = client.upload_files(
        files=files_to_upload,
        source_names=source_names,
        vault_id=vault_id
    )
    
    print("Upload initiated successfully!")
    print(f"Message: {upload_result.message}")
    print(f"Files processed: {upload_result.files_processed}")
    print(f"Task IDs: {upload_result.task_ids}")
    
    # Store task IDs for monitoring
    task_ids = upload_result.task_ids
else:
    print("No files to upload. Please add compatible files to the sample_docs directory.")


## Step 5: Monitor Upload Progress

Let's monitor the upload tasks to see when they complete:


In [ ]:
if 'task_ids' in locals() and task_ids:
    print("Monitoring upload tasks...")
    
    # Check status every 10 seconds for up to 3 minutes
    max_checks = 18  # 3 minutes (18 * 10 seconds)  
    checks = 0
    all_tasks_completed = False
    
    while not all_tasks_completed and checks < max_checks:
        all_tasks_completed = True
        completed_tasks = 0
        
        print(f"\nCheck #{checks + 1}:")
        
        for i, task_id in enumerate(task_ids, 1):
            task_status = client.get_task_status(task_id)
            state = task_status.get('state', 'UNKNOWN')
            status_msg = task_status.get('status', 'No status')
            
            if state in ['SUCCESS']:
                print(f"  Task {i}: {state} - {status_msg}")
                completed_tasks += 1
            elif state in ['FAILURE']:
                print(f"  Task {i}: {state} - {status_msg}")
                completed_tasks += 1
            else:
                print(f"  Task {i}: {state} - {status_msg}")
                all_tasks_completed = False
        
        print(f"Progress: {completed_tasks}/{len(task_ids)} tasks completed")
        
        if not all_tasks_completed:
            print("Waiting 10 seconds before next check...")
            time.sleep(10)
            checks += 1
        else:
            print("All tasks completed!")
    
    if checks >= max_checks:
        print("Maximum check time reached. Some tasks may still be processing.")
else:
    print("No tasks to monitor.")


## Step 6: Query the Vault

Once the documents are processed, let's query our vault to see what insights we can get:


In [ ]:
if 'all_tasks_completed' in locals() and all_tasks_completed:
    print("Querying the vault with processed documents...")
    
    query_result = client.query(
        query="How do the method perform against state of the art methods ?",
        vault_id=vault_id,
        top_k=5
    )
    
    print("Query Results:")
    print("="*50)
    
    # Display results using the built-in pretty print
    query_result.prettyprint()
    
else:
    print("Skipping query - documents may still be processing.")
    print("You can run a query manually once processing is complete.")
